# Bougival Dataset

## objectif

Est-ce que les données de détection des signatures acoustiques peuvent avertir d'un changement anomale de l'environement ?

Céation d'un dataset sur le site de Bougival pour étudier l'activité acoustique et la physico-chimie du fleuve. 
Identifier les éventuelles patterns de l'activité acoustique au cours du temps.   

Données à disposition:
- counting de signatures sonores 
  - Données yolo Sensea: 
    - **occurence de signatures acoustiques** => format (date,label)
  - Données AutoRiver:
    - **comptage de signatures acoustiques** => format (date, label1, ... , label10)
- **physico-chimie continue** journalière
  - oxygène, turbidité, conductivité, ph, phosphate, temperature
- **physico-chimie ponctuelle** hebdo faite en laboratoire
  - MES,DBO5,DCO,NTK,NH4,NO2,NO3,Pt,TAC,Chlorures,Sulfates,COD,PO4

Avec DBBuilder => Construire un dataset comprenant:
1) le comptage des détections faites de signatures acoustiques
2) l'ajout de varnamenamenamenamenamenamenamenamenamenamenamenamenamenameiables météorologiques
3) l'identification des cycles journaliers (jour, nuit, crépuscule, aube) et autres données astrales (phases de la lune)

In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

In [ ]:
import pandas as pd
import numpy as np
import glob
from DBBuilder import DBBuilder, Sun, Moon
from DBBuilder.datareader import Otoriver_DataReader, Sensea_DataReader
from IPython.display import display
import plotly.express as px

In [ ]:
# Physico-Chimie
files_phch_continus=glob.glob("./data/EB/bougival/chimie continue/*.csv")
files_phch_ponctual=glob.glob("./data/EB/bougival/chimie ponctuelle/*.csv")
# Outputs
output_folder=None

## Préparation des données acoustiques

In [ ]:
metadata_acoustic = pd.DataFrame()
# Variables
sitename="bougival"
lat, long, elev=48.865,2.144,23
timezone="UTC"
files_otoriver = glob.glob("./data/clean/bougival/acoustic/autoriver/*.csv")
output_folder = "./data/datasets/bougival/"

### Données otoriver (acoustic EB)

Note: Données otoriver
les colonnes on des types de valeurs differentes en fonction des colonnes => int / float => expl "B40k"=int "BUS"=float
Déduction = il n'y a pas que des données de comptage !! 
Si l'on regarde l'occurence des dates dans le jeu de données, on remarque que des dates sont jusqu'à 30 fois présentent.  

#### Première exploration des données

In [ ]:
df_otoriver = pd.concat([pd.read_csv(f) for f in files_otoriver])
display(df_otoriver)
df_unique_dates = df_otoriver.groupby("date").mean().reset_index()

In [ ]:
df_unique_dates

In [ ]:
df_otoriver.describe()


In [ ]:
df_otoriver.date = pd.to_datetime(df_otoriver.date)
dates_count=df_otoriver.date.value_counts()
dates_count = dates_count.sort_index()
display(px.line(df_otoriver, x="date", y=[c for c in df_otoriver.columns if c != "date"]))
display(px.line(x=dates_count.index, y=dates_count.values, title="Occurence des dates dans les données otoriver"))

#### Utilisation du datareader pour les données otoriver

In [ ]:
metadata_otoriver = pd.DataFrame(Otoriver_DataReader.metadata)
metadata_otoriver.to_csv("./data/metadata_otoriver.csv", index=False) 
display(metadata_otoriver)

In [ ]:
otoriver_reader = Otoriver_DataReader()
df_otoriver_prepared = otoriver_reader.from_df(df_otoriver)

In [ ]:
df_otoriver_prepared

In [ ]:
def get_data_byUnit (df:pd.DataFrame, metadata:pd.DataFrame, display_figs:bool=False):
    data_byUnit = {}
    for unit in metadata.unit.unique():
        labels_unit = metadata_otoriver[metadata_otoriver.unit == unit].varname.tolist()
        df_unit = None
        for var in labels_unit:
            df_var = df.loc[(df_otoriver[var].notna()),["date", var]]
            print(var)
            df_var = df_var.drop_duplicates()
            print(df_var.duplicated(subset="date").sum())
            if type(df_unit) == type(None):
                df_unit = df_var
            else:
                df_unit = pd.merge(df_unit, df_var, on="date", how="outer")
        fig = px.line(df_unit, x="date", y=labels_unit, title=f"unit {unit}")
        data_byUnit[unit] = {"df":df_unit, "fig":fig}
        if display_figs:
            display(fig)
    return data_byUnit

notes: 
Après consultation du fichier "Tableau des paramètres MeSeine.xlsx", les variables d'otoriver sont décrites et les unités de mesure données. 
On remarque que les variables n'ont pas la même unité, certaines sont en comptage (entier), mais d'autres sont en frequence ou pourcentage, notament pour les bioclicks.

=> Utilisation des données de type comptage

In [ ]:
var_count_otoriver = metadata_otoriver.loc[(metadata_otoriver.unit == "nb"), "varname"].values.tolist()
display(metadata_otoriver[metadata_otoriver.unit == "nb"])

In [ ]:
df_counting_otoriver = None
for var in var_count_otoriver:
    df_var = df_otoriver.loc[(df_otoriver[var].notna()),["date", var]]
    if type(df_counting_otoriver) == type(None):
        df_counting_otoriver = df_var
    else:
        df_counting_otoriver = pd.merge(df_counting_otoriver, df_var, on="date", how="outer")

In [ ]:
df_counting_otoriver

In [ ]:
labels_otoriver = [c for c in metadata_otoriver[metadata_otoriver.unit == "pct"].varname if c != "date"]
px.line(df_otoriver, x="date", y=labels_otoriver)

##  Données Sensea

In [ ]:
files_sensea = glob.glob("./data/clean/bougival/acoustic/sensea/*.csv")
df_sensea = pd.concat([pd.read_csv(f) for f in files_sensea])
display(df_sensea)

In [ ]:
file_lilian_sensea_clean = "data/seinacoustic_EB/1-bougival/données transformées/heure/acoustique/SENSEA/descripteurs150621270623.csv"
df_sensea_lilian = pd.read_csv(file_lilian_sensea_clean, sep=";")
display(df_sensea_lilian)

In [ ]:
dbbuilder = DBBuilder(latitude=lat, longitude=long, elevation=elev, timefreq="min")
dbbuilder.from_labels_occurence(dates=df_sensea.date, labels=df_sensea.label)
display(dbbuilder.counting)

In [ ]:
cols_label = [c for c in dbbuilder.counting.columns if c.startswith("label")]
counting_hourly = dbbuilder.agg_counting(timefreq="h")
counting_daily = dbbuilder.agg_counting(timefreq="d")

In [ ]:
display(counting_daily)

In [ ]:
px.line(dbbuilder.counting, x="date", y=[c for c in dbbuilder.counting.columns if c.startswith("label")])
count_hourly = db

In [ ]:
px.line(dbbuilder.counting, x="date", y=[c for c in dbbuilder.counting.columns if c.startswith("label")], )

In [ ]:
def plot_grpby (df, columns)

In [ ]:
# agg_params = {k:"mean" for k in dbbuilder.counting.columns if k.startswith("label")}
# agg_params.update({k:"var" for k in dbbuilder.counting.columns if k.startswith("label")})
hmean_suncycle_day = dbbuilder.counting.groupby(["suncycle_day"])[[c for c in dbbuilder.counting.columns if c.startswith("label")]].mean().add_prefix("hmean_")
hstd_suncycle_day = dbbuilder.counting.groupby(["suncycle_day"])[[c for c in dbbuilder.counting.columns if c.startswith("label")]].std().add_prefix("hstd_")
hmean_suncycle_day, hstd_suncycle_day = hmean_suncycle_day.reset_index(), hstd_suncycle_day.reset_index()
daily_analysis = pd.merge(hmean_suncycle_day, hstd_suncycle_day, on="suncycle_day")
display(daily_analysis)

In [ ]:
cols_mean = [c for c in daily_analysis.columns if c.startswith("hmean_label")]
cols_std = [c for c in daily_analysis.columns if c.startswith("hstd_label")]
px.histogram(daily_analysis, x="suncycle_day", y=cols_mean)

In [ ]:
output_sensea = dbbuilder.from_labels_occurence(dates=df_sensea.date, labels=df_sensea.label, prefix="label_sensea_")
sensea_database = {k:pd.DataFrame(v) for k,v in output_sensea["data"]}
for k, df in database:
    print(k)
    display(df)